In [35]:
#!pip install torch torchvision timm SimpleITK nibabel numpy scipy matplotlib scikit-learn monai ‘monai[einops]’
#!pip install git+https://github.com/DIAGNijmegen/Tiger.git@stable

import SimpleITK as sitk
import numpy as np
import torch
import torch.nn as nn
from timm.models.swin_transformer import SwinTransformer
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from monai.networks.nets import SwinUNETR
from tqdm import tqdm
import os
import multiprocessing as mp

In [2]:
#brew install wget
#brew install unzip
# !wget https://zenodo.org/api/records/10159290/files-archive -O data.zip
# !unzip data.zip
# !unzip data/images.zip -d data/extracted_images
# !unzip data/masks.zip -d data/extracted_masks

# Data Preprocessing

In [15]:
image_files = []

# Find all the files in the extracted directory
for root, _, files in os.walk('../../extracted_images/images'):
    for file in files:
        if file.endswith('.mha'):
            image_files.append(os.path.join(root, file))

In [16]:
mask_files = []

# Find all mask files in the extracted directory
for root, _, files in os.walk('../../extracted_images/masks'):
    for file in files:
        if file.endswith('.mha'):  # Assuming masks have "_mask" in their filename
            mask_files.append(os.path.join(root, file))

In [ ]:
image_mask_pairs = list(zip(image_files, mask_files))
for image_file, mask_file in image_mask_pairs:
    if os.path.basename(image_file) == os.path.basename(mask_file):
        continue
    else:
        print("mismatched files")

Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
S

In [55]:
def extract_padded_patches(volume, patch_size=(64, 64, 64), stride=(32, 32, 32)):
    """
    Extract overlapping patches from a 3D volume with padding for patches that exceed the volume boundary.
    Args:
        volume (numpy array): Input 3D volume (depth, height, width).
        patch_size (tuple): Size of the patch (depth, height, width).
        stride (tuple): Stride for sliding window extraction.
    Returns:
        list: List of padded 3D patches.
    """
    depth, height, width = volume.shape
    patch_depth, patch_height, patch_width = patch_size
    stride_d, stride_h, stride_w = stride

    patches = []

    for z in range(0, depth, stride_d):
        for y in range(0, height, stride_h):
            for x in range(0, width, stride_w):
                # Extract patch and pad if necessary
                patch = volume[z:z + patch_depth, y:y + patch_height, x:x + patch_width]
                padded_patch = pad_volume(patch, patch_size)
                patches.append(padded_patch)

    return patches


class MRIDatasetWithPatches(Dataset):
    def __init__(self, image_mask_pairs, target_size=(64, 64, 64), stride=(32, 32, 32), transform=None):
        """
        Dataset class for extracting padded patches.
        Args:
            image_mask_pairs (list): List of (image_path, mask_path) pairs.
            target_size (tuple): Patch size for extraction.
            stride (tuple): Stride for overlapping patches.
            transform (callable, optional): Transformations to apply to the data.
        """
        self.image_mask_pairs = image_mask_pairs
        self.target_size = target_size
        self.stride = stride
        self.transform = transform

    def __len__(self):
        return len(self.image_mask_pairs)

    def __getitem__(self, idx):
        image_file, mask_file = self.image_mask_pairs[idx]

        # Load image and mask
        image = sitk.ReadImage(image_file)
        mask = sitk.ReadImage(mask_file)

        # Convert to numpy arrays
        image = sitk.GetArrayFromImage(image)
        mask = sitk.GetArrayFromImage(mask)

        # Extract padded patches
        image_patches = extract_padded_patches(image, self.target_size, self.stride)
        mask_patches = extract_padded_patches(mask, self.target_size, self.stride)

        # Select a specific patch based on idx
        patch_idx = idx % len(image_patches)  # Cycle through available patches
        img_patch = image_patches[patch_idx]
        mask_patch = mask_patches[patch_idx]

        # Apply transformations (optional)
        if self.transform:
            img_patch, mask_patch = self.transform(img_patch, mask_patch)

        # Convert to PyTorch tensors
        img_tensor = torch.tensor(img_patch, dtype=torch.float32).unsqueeze(0)  # Add channel dimension
        mask_tensor = torch.tensor(mask_patch, dtype=torch.long)

        return img_tensor, mask_tensor


def pad_volume(volume, target_size):
    """
    Pads a 3D volume to the target size.
    Args:
        volume (numpy array): Input 3D volume.
        target_size (tuple): Target size (depth, height, width).
    Returns:
        numpy array: Padded volume.
    """
    depth, height, width = volume.shape
    target_depth, target_height, target_width = target_size

    pad_depth = max(0, target_depth - depth)
    pad_height = max(0, target_height - height)
    pad_width = max(0, target_width - width)

    padding = [
        (0, pad_depth),  # Pad at the end only
        (0, pad_height),
        (0, pad_width),
    ]

    return np.pad(volume, padding, mode="constant", constant_values=0)

In [28]:
class MRIDataset(Dataset):
    def __init__(self, image_mask_pairs, target_size=(128, 128, 128), transform=None):
        """
        Initializes the MRIDataset.

        Args:
            image_mask_pairs (list of tuples): List of (image_path, mask_path) pairs.
            target_size (tuple): Desired size for the resampled images and masks (depth, height, width).
            transform (callable, optional): Transformations to apply to the data.
        """
        self.image_mask_pairs = image_mask_pairs
        self.target_size = target_size
        self.transform = transform

    def __len__(self):
        return len(self.image_mask_pairs)

    def __getitem__(self, idx):
        image_file, mask_file = self.image_mask_pairs[idx]

        # Load image and mask
        image = sitk.ReadImage(image_file)
        mask = sitk.ReadImage(mask_file)

        # Resample image and mask
        image = self.resample_image(image, self.target_size, sitk.sitkLinear)
        mask = self.resample_image(mask, self.target_size, sitk.sitkNearestNeighbor)

        # Convert to numpy arrays
        image = sitk.GetArrayFromImage(image)
        mask = sitk.GetArrayFromImage(mask)

        # Normalize the image
        image = (image - np.min(image)) / (np.max(image) - np.min(image))
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Add channel dimension
        mask = torch.tensor(mask, dtype=torch.long)  # Long type for classification tasks

        if self.transform:
            image, mask = self.transform(image, mask)

        return image, mask

    @staticmethod
    def resample_image(image, target_size, interpolator):
        """
        Resamples a 3D image to the target size.

        Args:
            image (SimpleITK.Image): The input image.
            target_size (tuple): Desired size (depth, height, width).
            interpolator (SimpleITK.Interpolator): Interpolation method (e.g., sitk.sitkLinear or sitk.sitkNearestNeighbor).

        Returns:
            SimpleITK.Image: Resampled image.
        """
        original_size = image.GetSize()
        original_spacing = image.GetSpacing()

        # Calculate target spacing
        target_spacing = [
            (original_size[i] * original_spacing[i]) / target_size[i]
            for i in range(3)
        ]

        # Resample the image
        resample = sitk.ResampleImageFilter()
        resample.SetOutputSpacing(target_spacing)
        resample.SetSize(target_size)
        resample.SetInterpolator(interpolator)
        resample.SetOutputOrigin(image.GetOrigin())
        resample.SetOutputDirection(image.GetDirection())
        resample.SetDefaultPixelValue(0)

        return resample.Execute(image)

In [31]:
dataset = MRIDatasetWithPatches(image_mask_pairs=image_mask_pairs)

In [32]:
# Split image-mask pairs into training and validation
train_pairs, val_pairs = train_test_split(image_mask_pairs, test_size=0.2, random_state=55)

# Create datasets
train_dataset = MRIDatasetWithPatches(image_mask_pairs=train_pairs)
val_dataset = MRIDatasetWithPatches(image_mask_pairs=val_pairs)

# 3D Swin Transformer

In [36]:
class Swin3DSegmentation(nn.Module):
    def __init__(self, input_channels=1, num_classes=210, img_size=(64, 64, 64), embed_dim=96, patch_size=4):
        """
        3D segmentation model using SwinUNETR backbone.
        Args:
            input_channels (int): Number of input channels (e.g., 1 for grayscale MRI images).
            num_classes (int): Number of output classes for segmentation.
            img_size (tuple): Size of the 3D input image (D, H, W).
            embed_dim (int): Embedding dimension of the Swin Transformer.
            patch_size (int): Patch size for splitting the input volume.
        """
        super(Swin3DSegmentation, self).__init__()
        
        # SwinUNETR backbone
        self.swin_unetr = SwinUNETR(
            img_size=img_size,  # 3D image size
            in_channels=input_channels,
            out_channels=num_classes,
            feature_size=embed_dim,
            use_checkpoint=True  # Enables gradient checkpointing for memory efficiency
        )

    def forward(self, x):
        """
        Forward pass for 3D segmentation.
        Args:
            x (torch.Tensor): Input tensor of shape (B, C, D, H, W).
                              - B: Batch size
                              - C: Number of input channels
                              - D: Depth of the input image
                              - H: Height of the input image
                              - W: Width of the input image
        Returns:
            torch.Tensor: Output tensor of shape (B, num_classes, D, H, W).
                          - B: Batch size
                          - num_classes: Number of segmentation classes
                          - D: Depth of the output image
                          - H: Height of the output image
                          - W: Width of the output image
        """
        return self.swin_unetr(x)

def save_model(model, path):
    """
    Saves the model's state dictionary to the specified path.

    Args:
        model: The trained model to be saved.
        path: The file path where the model will be saved.
    """
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

def train(model, train_loader, val_loader, epochs, device, save_path="trained_model.pth"):
    """
    Trains the model on the given data loaders and saves it after training.
    Args:
        model: The neural network model to be trained.
        train_loader: DataLoader for the training data.
        val_loader: DataLoader for the validation data.
        epochs: Number of epochs to train.
        device: Device for training (e.g., 'cpu', 'cuda', 'mps').
        save_path: Path to save the trained model.
    """
    # Define loss function and optimizer
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        epoch_loss = 0.0
        print(f"Epoch {epoch + 1}/{epochs}")

        # Training loop with progress bar
        for images, masks in tqdm(train_loader, desc="Training", leave=False):
            images, masks = images.to(device), masks.to(device)  # Move data to device

            optimizer.zero_grad()  # Zero the gradients
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, masks)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights

            epoch_loss += loss.item()

        avg_train_loss = epoch_loss / len(train_loader)
        print(f"Training Loss: {avg_train_loss:.4f}")

        # Validation loop
        model.eval()  # Set the model to evaluation mode
        val_loss = 0.0
        with torch.no_grad():  # Disable gradient calculations for validation
            for images, masks in val_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)
                loss = criterion(outputs, masks)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        print(f"Validation Loss: {avg_val_loss:.4f}")
        
    # Save the model after training
    save_model(model, save_path)

# Training

In [46]:
mp.set_start_method('spawn', force=True)  # Ensure proper start method for multiprocessing
# Check device compatibility
# device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=8, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=4, pin_memory=True)

# Initialize the model and move it to the appropriate device
model = Swin3DSegmentation(input_channels=1, num_classes=210).to(device)
images, masks = next(iter(train_loader))
images = images.to(device)
train(model, train_loader, val_loader, epochs=10, device=device, save_path="patch_model.pth")

PicklingError: Can't pickle <class '__main__.MRIDatasetWithPatches'>: it's not the same object as __main__.MRIDatasetWithPatches

# Evaluation

In [62]:
# Define the model architecture (match the training setup)
model = Swin3DSegmentation(input_channels=1, num_classes=210).to(device)

# Load the state dictionary and map to CPU
state_dict = torch.load("patch_model.pth", map_location="cpu")

# Load the weights into the model
model.load_state_dict(state_dict, strict = False)

# Set the model to evaluation mode
model.eval()

/opt/homebrew/lib/python3.11/site-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
  warn_deprecated(argname, msg, warning_category)
/var/folders/kh/06qpp3c15bn4xjm4k3qs643c0000gn/T/ipykernel_87041/3232614033.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects wi

Swin3DSegmentation(
  (swin_unetr): SwinUNETR(
    (swinViT): SwinTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv3d(1, 96, kernel_size=(2, 2, 2), stride=(2, 2, 2))
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (layers1): ModuleList(
        (0): BasicLayer(
          (blocks): ModuleList(
            (0-1): 2 x SwinTransformerBlock(
              (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (attn): WindowAttention(
                (qkv): Linear(in_features=96, out_features=288, bias=True)
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=96, out_features=96, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (softmax): Softmax(dim=-1)
              )
              (drop_path): Identity()
              (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (mlp): MLPBlock(
                (linear1): Linear(in_featur

In [70]:
from sklearn.metrics import jaccard_score

# Example Dice coefficient function
def dice_coefficient(preds, targets, smooth=1e-6):
    """
    Calculate the Dice Coefficient.
    Args:
        preds (torch.Tensor): Predicted binary or multi-class mask (logits or probabilities).
        targets (torch.Tensor): Ground truth mask (same shape as preds).
        smooth (float): Smoothing factor to avoid division by zero.
    Returns:
        float: Dice score.
    """
    # Ensure predictions are binary (e.g., logits converted to binary masks)
    preds = (preds > 0.5).float()
    intersection = torch.sum(preds * targets)
    union = torch.sum(preds) + torch.sum(targets)
    dice = (2.0 * intersection + smooth) / (union + smooth)
    return dice.item()

# Load the dataset
# Replace 'MRIDatasetWithPatches' and 'test_image_mask_pairs' with your dataset class and test data
test_dataset = MRIDatasetWithPatches(
    image_mask_pairs=val_pairs,  # Test dataset pairs
    target_size=(64, 64, 64),                 # Replace with your patch size
    stride=(32, 32, 32)                      # Replace with your stride
)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=0)

# Run evaluation
dice_scores = []

In [ ]:
from torch.nn import functional as F

dice_scores = []

with torch.no_grad():  # Disable gradient calculations for evaluation
    for images, masks in test_loader:
        images, masks = images.to(device), masks.to(device)  # Move to device

        # Get predictions
        outputs = model(images)  # Model output: logits or probabilities
        # print(f"Output shape: {outputs.shape}")

        probs = F.softmax(outputs, dim=1)  # Convert logits to probabilities (multi-class)
        preds = torch.argmax(probs, dim=1)  # Get predicted class for each pixel
        # print(f"Predicted classes: {torch.unique(preds)}")

        # Calculate Dice coefficient for each valid class
        valid_classes = torch.unique(masks)
        # print(f"Valid classes in ground truth: {valid_classes}")
        
        for class_idx in valid_classes:
            if class_idx == 0:  # Skip background
                continue
            dice = dice_coefficient((preds == class_idx).float(), (masks == class_idx).float())
            dice_scores.append((class_idx.item(), dice))

# Calculate average Dice score
class_dice_scores = {class_idx: [] for class_idx, _ in dice_scores}
for class_idx, dice in dice_scores:
    class_dice_scores[class_idx].append(dice)

average_dice_scores = {class_idx: np.mean(scores) for class_idx, scores in class_dice_scores.items()}

# Print the results
print("Dice Scores by Class:")
for class_idx, dice_score in average_dice_scores.items():
    print(f"Class {class_idx}: {dice_score:.4f}")